In [2]:
import re
import os
import numpy as np
import pandas as pd
import string
from matplotlib import pyplot as plt

In [3]:
def load_train_test_imdb_data(data_dir):
    """Loads the IMDB train/test datasets from a folder path.
    Input:
    data_dir: path to the "aclImdb" folder.

    Returns:
    train/test datasets as pandas dataframes.
    """

    data = {}
    for split in ["train", "test"]:
        data[split] = []
        for sentiment in ["neg", "pos"]:
            score = 1 if sentiment == "pos" else 0

            path = os.path.join(data_dir, split, sentiment)
            file_names = os.listdir(path)
            for f_name in file_names:
                with open(os.path.join(path, f_name), "r",
                encoding="utf-8") as f:
                    review = f.read()
                    data[split].append([review, score])

    # CODE CỦA BẠN Ở ĐÂY
    # Xáo trộn dữ liệu huấn luyện sử dụng "np.random.shuffle":
    np.random.shuffle(data["train"])

    # Tạo khung dữ liệu huấn luyện "data["train"])" chỉ có 2 cột: "text" và "sentiment"
    data["train"] = pd.DataFrame(data["train"], columns=["text", "sentiment"])

    # Xáo trộn dữ liệu kiểm tra sử dụng "np.random.shuffle":
    np.random.shuffle(data["test"])

    # Tạo khung dữ liệu kiểm tra "data["test"])" chỉ có 2 cột: "text" và "sentiment"
    data["test"] = pd.DataFrame(data["test"], columns=["text", "sentiment"])

    return data["train"], data["test"]

In [4]:
train_data, test_data = load_train_test_imdb_data("data/aclImdb")

In [5]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """

    # loại bỏ HTML tag
    text = re.sub(r'<.*?>', '', text)

    # CODE CỦA BẠN Ở ĐÂY
    # loại bỏ các ký tự [\], ['] và ["] bằng phương thức resub:
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    # CODE CỦA BẠN Ở ĐÂY

    # chuyển đổi văn bản thành chữ thường
    text = text.strip().lower()

    # thay dấu chấm bằng dấu cách
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

# Ví dụ
clean_text("<html>This <br> is  [\] [\'] [\"] not, a sentence.<\html>").split()

['this', 'is', 'not', 'a', 'sentence']

In [6]:
train_data.head()

,text,sentiment
0,After seeing only half of the film in school b...,0
1,"In this paranoia-driven potboiler, our reporte...",0
2,That's the sound of Stan and Ollie spinning in...,0
3,This film could be one of the most underrated ...,1
4,"Time and time again, I've stated that if peopl...",1


In [7]:
# write clean text to a new file
with open("data/imdb_train_clean.txt", "w", encoding="utf-8") as f:
    for text, score in train_data.values:
        f.write("{} {}\n".format(clean_text(text), score))

In [8]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


# Chuyển mỗi văn bản thành một vectơ đếm từ
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             ngram_range=(1, 2))

training_features = vectorizer.fit_transform(train_data["text"])
test_features = vectorizer.transform(test_data["text"])

# CODE CỦA BẠN Ở ĐÂY
# Huấn luyện
# Gán "LinearSVC()" vào một biến mô hình:
model = LinearSVC()

# Khớp mô hình với dữ liệu huấn luyện "training_features" và nhãn huấn luyện "train_data["sentiment"]":
model.fit(training_features, train_data["sentiment"])

# Dự đoán dữ liệu kiểm tra "test_features":
y_test_pred = model.predict(test_features)
# CODE CỦA BẠN Ở ĐÂY

# Đánh giá
acc = accuracy_score(test_data["sentiment"], y_test_pred)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

Accuracy on the IMDB dataset: 88.66


In [9]:
from IPython.core.display_functions import display
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=1234)
param_dict = {'C': [0.001, 0.01, 0.1, 1, 10],
             'solver': ['sag', 'lbfgs', 'saga']}
grid_search = GridSearchCV(lr_model, param_dict)
grid_search.fit(training_features, train_data["sentiment"])
display(grid_search.best_params_)
print("Cross-Validated Score of the Best Estimator: %.3f" % grid_search.best_score_)

/Users/ngohongthai/Documents/projects/tensorflow-env/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'C': 10, 'solver': 'saga'}

Cross-Validated Score of the Best Estimator: 0.893
